<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/structured_40%E2%80%91D_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf

# Reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

def make_wormhole_data(n_samples=12000, n_features=40, n_classes=10,
                       turns=3.0, noise=0.12, seed=SEED):
    """
    Build 10 phase-shifted logarithmic spirals in 2D and project to 40D.
    This creates a 'tunneled' manifold with real class structure.
    """
    rng = np.random.default_rng(seed)
    per_class = n_samples // n_classes

    # 2D -> n_features random projection (fixed across classes for shared geometry)
    proj = rng.normal(0.0, 1.0, size=(2, n_features))

    xs, ys = [], []
    for c in range(n_classes):
        t = rng.uniform(0.0, 1.0, size=(per_class,))  # progress along the spiral
        theta = 2 * np.pi * turns * t + (2 * np.pi / n_classes) * c
        r = 0.8 + 1.2 * t  # outward growth
        x2 = np.stack([r * np.cos(theta), r * np.sin(theta)], axis=1)
        x_high = x2 @ proj + rng.normal(0.0, noise, size=(per_class, n_features))
        xs.append(x_high.astype("float32"))
        ys.append(np.full((per_class,), c, dtype=np.int32))

    X = np.vstack(xs)
    y = np.concatenate(ys)

    # Shuffle
    idx = rng.permutation(len(X))
    return X[idx], y[idx]

# Generate data
X, y = make_wormhole_data(n_samples=12000, n_features=40, n_classes=10, turns=3.5, noise=0.12)

# Split: 70/15/15
n = len(X)
n_train = int(0.70 * n)
n_val   = int(0.15 * n)
X_train, y_train = X[:n_train], y[:n_train]
X_val, y_val     = X[n_train:n_train+n_val], y[n_train:n_train+n_val]
X_test, y_test   = X[n_train+n_val:], y[n_train+n_val:]

# Standardize by train stats (important for tanh/relu stability)
mean = X_train.mean(axis=0, keepdims=True)
std  = X_train.std(axis=0, keepdims=True) + 1e-7
X_train = (X_train - mean) / std
X_val   = (X_val   - mean) / std
X_test  = (X_test  - mean) / std

# tf.data pipelines
BATCH = 128
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(8192, seed=SEED).batch(BATCH).prefetch(tf.data.AUTOTUNE)
val_ds   = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(BATCH).prefetch(tf.data.AUTOTUNE)
test_ds  = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH).prefetch(tf.data.AUTOTUNE)

# Wormhole Neural Network Model (with light regularization)
class WormholeAI(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.entry_layer  = tf.keras.layers.Dense(256, activation='tanh',
                                                  kernel_regularizer=tf.keras.regularizers.l2(1e-5))
        self.tunnel_layer = tf.keras.layers.Dense(512, activation='relu',
                                                  kernel_regularizer=tf.keras.regularizers.l2(1e-5))
        self.dropout      = tf.keras.layers.Dropout(0.2)
        self.exit_layer   = tf.keras.layers.Dense(256, activation='tanh',
                                                  kernel_regularizer=tf.keras.regularizers.l2(1e-5))
        self.output_layer = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, inputs, training=False):
        x = self.entry_layer(inputs)        # Enter the wormhole
        x = self.tunnel_layer(x)            # Information tunnels
        x = self.dropout(x, training=training)
        x = self.exit_layer(x)              # Exit the wormhole
        return self.output_layer(x)         # Class projection

model = WormholeAI()
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks and logging
os.makedirs("results", exist_ok=True)
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-5, verbose=1),
    tf.keras.callbacks.CSVLogger("results/training_history.csv"),
    tf.keras.callbacks.ModelCheckpoint("results/wormhole_best.keras", monitor="val_accuracy",
                                       save_best_only=True, save_weights_only=False)
]

history = model.fit(train_ds, validation_data=val_ds, epochs=60, callbacks=callbacks, verbose=1)

# Evaluate
test_loss, test_acc = model.evaluate(test_ds, verbose=0)
print(f"✅ Test accuracy: {test_acc:.4f} | Test loss: {test_loss:.4f}")
print("✅ Training history saved to results/training_history.csv")
print("✅ Best model saved to results/wormhole_best.keras")

# Confusion matrix (quick look at per-class performance)
y_pred = model.predict(test_ds, verbose=0).argmax(axis=1)
cm = tf.math.confusion_matrix(y_test, y_pred, num_classes=10).numpy()
print("Confusion matrix:\n", cm)